<a href="https://colab.research.google.com/github/d-smit/Deep_Learning_Work/blob/master/road_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [137]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

import os 
import glob 
import shutil
from pathlib import Path
import random
import cv2
import matplotlib.pyplot as plt 
import numpy as np
from PIL import Image
from keras.preprocessing.image import load_img

Mounted at /content/gdrive


## Set Data Paths

In [0]:
roads = '/content/gdrive/My Drive/Colab Notebooks/aerolab_data/road_dataset'

In [0]:
path_to_gt = roads + '/targets/'
path_to_imgs = roads + '/source/'

## Sort Directories

Large number of files mean high amount of memory required to read all at once. If we sort into train/test directories, we can use a generator to instead load images in batches during training. 


In [0]:
splits = ['train/', 'val/', 'test/']
dirs = [path_to_gt, path_to_imgs]

for data in dirs:
  for split in splits:
    if not os.path.isdir(os.path.join(data, split)):
            os.makedirs(os.path.join(data, split))

In [150]:
imgs = sorted(os.listdir(path_to_imgs))
masks = sorted(os.listdir(path_to_gt))

if len(imgs) != len(masks):
  diffs = set(imgs) ^ set(masks)
  imgs = [img for img in imgs if img not in diffs]

imgs = imgs[:-3]
masks = masks[:-3]

print(imgs[-5:])
print(masks[-5:])

pairs = list(zip(imgs, masks))
random.shuffle(pairs)
imgs, masks = zip(*pairs)

['Zachary_6.TIF', 'Zachary_7.TIF', 'Zebulon_2.TIF', 'Zebulon_5.TIF', 'Zebulon_9.TIF']
['Zachary_6.TIF', 'Zachary_7.TIF', 'Zebulon_2.TIF', 'Zebulon_5.TIF', 'Zebulon_9.TIF']


In [0]:
train_split = int(0.6 * len(imgs))
val_split = int(0.8 * len(imgs))

In [0]:
train_imgs = imgs[:train_split]
val_imgs = imgs[train_split:val_split]
test_imgs = imgs[val_split:]

In [0]:
train_masks = masks[:train_split]
val_masks = masks[train_split:val_split]
test_masks = masks[val_split:]

In [0]:
def sort_data(data):

  for file in os.listdir(data):
          try:
            if os.path.isdir(file):
              continue
            if file in train_imgs or train_masks:
              shutil.copy2(os.path.join(data,file), os.path.join(data, split[0]))
            elif file in val_imgs or val_masks:
              shutil.copy2(os.path.join(data,file), os.path.join(data, split[1]))
            elif file in test_imgs or test_masks:
              shutil.copy2(os.path.join(data,file), os.path.join(data, split[2]))
          except IsADirectoryError:
            continue 
  

sort_data(path_to_imgs)